<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
# !pip install datasets

Импорт необходимых библиотек

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

/Users/antoneremin/Documents/JupyterLab/python_dir/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/antoneremin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
if torch.backends.mps.is_available():
    device = 'mps'  # MPS (Metal Performance Shaders) для Apple Silicon
elif torch.cuda.is_available():
    device = 'cuda'  # Для CUDA
else:
    device = 'cpu'  # CPU fallback

device

'mps'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [5]:
# Загрузим датасет
dataset = load_dataset('imdb')

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предложения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [35]:
sentences = []
word_threshold = 32

# Получить отдельные предложения и поместить их в sentences
for sentence in tqdm(dataset['unsupervised']['text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='english') if len(x) < word_threshold]
        )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:03<00:00, 12922.76it/s]


In [36]:
# Посмотрим на случайные примеры предложений из датасета
import random

for snt in random.sample(sentences, 5):
    print(snt)

you cannot go.
he will be missed!
beautiful.
there power is too great.
got the ending figured out yet?


In [38]:
print("Всего предложений:", len(sentences))

Всего предложений: 55345


Посчитаем для каждого слова его встречаемость.

In [39]:
from nltk.tokenize import word_tokenize

words = Counter()

# Расчет встречаемости слов
for sentence in tqdm(sentences):
    for word in word_tokenize(sentence):
        words[word] += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55345/55345 [00:01<00:00, 37932.76it/s]


In [40]:
len(words)

14257

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [44]:
vocab = set()
vocab_size = 10000

# Наполнение словаря
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])

for word, cnt in words.most_common(vocab_size):
    vocab.add(word)

In [45]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [46]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 10004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [47]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [70]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        tokenized_sentence = [self.bos_id]
        
        # Допишите код здесь
        for word in self.data[idx]:
            tokenized_sentence.append(word2ind.get(word, self.unk_id))
        tokenized_sentence.append(self.eos_id)
        
        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [71]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [72]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [73]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            # logits = # Посчитайте логиты предсказаний следующих слов
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [115]:
def train_model(model, optimizer, criterion, train_dataloader, eval_dataloader, num_epoch=20):
    # Напишите код здесь

    losses = []
    perplexities = []
    
    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()
        for batch in tqdm(train_dataloader, desc=f'Training epoch {epoch}:'):
            optimizer.zero_grad()
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(
                logits, batch['target_ids'].flatten())
            loss.backward()
            optimizer.step()
    
            epoch_losses.append(loss.item())
        
        losses.append(sum(epoch_losses) / len(epoch_losses))
        perplexities.append(evaluate(model, criterion, eval_dataloader))

    return {'losses': losses, 'perplexities': perplexities, 'model': model}

### Первый эксперимент (2 балла)

В качестве самой базовой архитектуры возьмем архитектуру на GRU слое. В качестве регуляризации будем использовать только Dropout с вероятностью
0.1. После каждого эксперимента будем оценивать только по 1 параметру это значение метрики перплексии на последней эпохи.

In [117]:
class BaseGRU(nn.Module):

    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [118]:
model = BaseGRU(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:24<00:00, 13.97it/s]


In [119]:
# Итоговое значение перплексии для первого эксперимента:
experiment_1 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_1)

3.663148244954474


### Второй эксперимент (2 балла)

Поменяем только слой с GRU на LSTM и сравним значение метрики. Дальше будем пробовать улучшать именно ту модель, которая покажет лучшую метрику.

In [120]:
class BaseLSTM(nn.Module):

    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [121]:
model = BaseLSTM(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:17<00:00, 19.95it/s]


In [122]:
# Итоговое значение перплексии для второго эксперимента:
experiment_2 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_2)

3.772892312520111


# Итог по базовой модели LSTM

Как мы видим, но просто замена слоя с GRU на LSTM не дала никакого прироста, более того, данная модель стала дольше обучаться (что в целом логично, так как сам слой LSTM сложнее слоя GRU).

# Эксперимент 3

Попробуем увеличить модель LSTM, установив для неё кол-во слоев равное 3.

In [123]:
class BaseLSTM_more_layers(nn.Module):

    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, num_layers = 3,  batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [124]:
model = BaseLSTM_more_layers(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:22<00:00, 15.05it/s]


In [125]:
# Итоговое значение перплексии для третьего эксперимента:
experiment_3 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_3)

3.8253247265442174


# Итог по модели LSTM с параметром num_layers = 3

Как мы видим, значение перплексии оказалось чуть хуже чем в предыдущем примере, поэтому оставляем версию со слоем GRU и пробуем в следующих экспериментах только её улучшать

# Эксперимент 4

Для сети со слоем GRU изменяем значение Dropout с 0.1 до 0.3.

In [126]:
class BaseGRU_inrease_dropout(nn.Module):

    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [127]:
model = BaseGRU_inrease_dropout(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:25<00:00, 13.48it/s]


In [128]:
# Итоговое значение перплексии для четвертого эксперимента:
experiment_4 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_4)

3.8055019389649143


# Эксперимент 5

Добавляем слой BatchNorm

In [129]:
class BaseGRU_with_batchnorm(nn.Module):

    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        
        batch_size, seq_len, hidden_dim = output.size()
        output = output.contiguous().view(-1, hidden_dim)
        output = self.batch_norm(output)
        output = output.view(batch_size, seq_len, hidden_dim)
        
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [130]:
model = BaseGRU_with_batchnorm(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:26<00:00, 13.20it/s]


In [131]:
# Итоговое значение перплексии для пятого эксперимента:
experiment_5 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_5)

3.4351404020863194


# Эксперимент 6

Добавляем слой LayerNorm

In [132]:
class BaseGRU_with_layer_norm(nn.Module):

    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        output = self.linear(output)  # Применяем линейный слой
        output = self.layer_norm(output)  # Применяем LayerNorm
        
        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.non_lin(output))  # [batch_size, seq_len, vocab_size]

        return projection

In [133]:
model = BaseGRU_with_layer_norm(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:24<00:00, 14.33it/s]


In [134]:
# Итоговое значение перплексии для шестого эксперимента:
experiment_6 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_6)

3.3857570903092484


# Эксперимент 7

Пробуем SGQ вместо Adam

In [135]:
model = BaseGRU_with_layer_norm(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.SGD(model.parameters())

train_results = train_model(model=model, 
                            optimizer=optimizer, 
                            criterion=criterion, 
                            train_dataloader=train_dataloader, 
                            eval_dataloader=eval_dataloader)

Training epoch 19:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 346/346 [00:21<00:00, 15.76it/s]


In [136]:
# Итоговое значение перплексии для седьмого эксперимента:
experiment_7 = evaluate(train_results["model"], criterion=criterion, dataloader=test_dataloader)
print(experiment_7)

18.71813085002284


### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.

# Итоговый отчет

Было проведено 7 экспериментов. Были протестированы различные архитектуры сетей GRU / LSTM. Также исследовалось влияние кол-во слоев на итоговое качество модели, а также изменялась величина Dropout. Использовались техники Layer Norm и Batch Norm, а также различные алгоритмы изменения learning rate. По итогу лучшее качество было достигнуто на сети со следующими параметраметрами базовый слой GRU, с дропаутом = 0.1, а также LayerNorm и Adam в качестве алгоритма оптимизации. Более подробно отражено в таблице ниже.

| Номер эксперимента | Базовый слой сети   | Слой нормализации | Величина Дропаута | Алгоритм Оптимизации | Полученное значение Перплексии |
|:------------------:|:-------------------:|:-----------------:|:-----------------:|:--------------------:|:------------------------------:|
| 1                  | GRU                 | Без нормализации  | 0.1               | Adam                 | 3.663                          |
| 2                  | LSTM                | Без нормализации  | 0.1               | Adam                 | 3.772                          |
| 3                  | LSTM (num_layers=3) | Без нормализации  | 0.1               | Adam                 | 3.825                          |
| 4                  | GRU                 | Без нормализации  | 0.3               | Adam                 | 3.805                          |
| 5                  | GRU                 | Batch Norm        | 0.1               | Adam                 | 3.435                          |
| **6**              | **GRU**             | **Layer Norm**    | **0.1**           | **Adam**             | **3.385**                      |
| 7                  | GRU                 | Layer Norm        | 0.1               | SGD                  | 18.718                         |
